In [1]:
import gc
from functools import partial

import torch

from diffusers import IFPipeline, IFSuperResolutionPipeline, StableDiffusionUpscalePipeline
from wandb_addons.diffusers import IFCallback

In [2]:
pipeline_1 = IFPipeline.from_pretrained(
    "DeepFloyd/IF-I-XL-v1.0", variant="fp16", torch_dtype=torch.float16
)
pipeline_1.enable_model_cpu_offload()


A mixture of fp16 and non-fp16 filenames will be loaded.
Loaded fp16 filenames:
[text_encoder/model.fp16-00002-of-00002.safetensors, unet/diffusion_pytorch_model.fp16.safetensors, text_encoder/model.fp16-00001-of-00002.safetensors, safety_checker/model.fp16.safetensors]
Loaded non-fp16 filenames:
[watermarker/diffusion_pytorch_model.safetensors
If this behavior is not expected, please check your folder structure.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
prompt = 'a photo of a smiling bee wearing a yellow hoodie and blue sunglasses standing in front of the eiffel tower holding a sign that says "Weights and Biases"'
prompt_embeds, negative_embeds = pipeline_1.encode_prompt(prompt)
num_images_per_prompt = 2
num_inference_steps = 100
configs = {"guidance_scale": 7.0}

callback = IFCallback(
    pipeline=pipeline_1,
    prompt=prompt,
    wandb_project="diffusers-2",
    wandb_entity="geekyrakshit",
    weave_mode=False,
    num_inference_steps=num_inference_steps,
    num_images_per_prompt=num_images_per_prompt,
    configs=configs
)

image = pipeline_1(
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=negative_embeds,
    output_type="pt",
    callback=partial(callback, end_experiment=False),
    **configs,
).images

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: geekyrakshit. Use `wandb login --relogin` to force relogin


  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
pipeline_2 = IFSuperResolutionPipeline.from_pretrained(
    "DeepFloyd/IF-II-L-v1.0", text_encoder=None, variant="fp16", torch_dtype=torch.float16
)
pipeline_2.enable_model_cpu_offload()


A mixture of fp16 and non-fp16 filenames will be loaded.
Loaded fp16 filenames:
[text_encoder/model.fp16-00002-of-00002.safetensors, unet/diffusion_pytorch_model.fp16.safetensors, text_encoder/model.fp16-00001-of-00002.safetensors, safety_checker/model.fp16.safetensors]
Loaded non-fp16 filenames:
[watermarker/diffusion_pytorch_model.safetensors
If this behavior is not expected, please check your folder structure.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [5]:
num_inference_steps = 50

callback.add_stage(pipeline_2, num_inference_steps=num_inference_steps)

image = pipeline_2(
    image=image,
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=negative_embeds,
    num_inference_steps=num_inference_steps,
    output_type="pt",
    callback=partial(callback, end_experiment=False),
).images

  0%|          | 0/50 [00:00<?, ?it/s]

In [6]:
del pipeline_2
gc.collect()
torch.cuda.empty_cache()

In [7]:
safety_modules = {
    "feature_extractor": pipeline_1.feature_extractor,
    "safety_checker": pipeline_1.safety_checker,
    "watermarker": pipeline_1.watermarker,
}
pipeline_3 = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler", **safety_modules, torch_dtype=torch.float16
)
pipeline_3.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
num_inference_steps = 75

callback.add_stage(pipeline_3, num_inference_steps=num_inference_steps, stage_name="Upscale")

image = pipeline_3(
    prompt=prompt,
    image=image,
    noise_level=100,
    num_inference_steps=num_inference_steps,
    callback=callback,
).images

/opt/conda/lib/python3.10/site-packages/diffusers/image_processor.py:310: FutureWarning: Passing `image` as torch tensor with value range in [-1,1] is deprecated. The expected value range for image tensor is [0,1] when passing as pytorch tensor or numpy Array. You passed `image` with value range [-1.0,1.0]
  warnings.warn(


  0%|          | 0/75 [00:00<?, ?it/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
